<a href="https://colab.research.google.com/github/Damanger/COLAB_CF/blob/main/BERT_NER_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtener cuenta en hugging face

Y generar un token, agregarlo a Secrets (secretos , la llave) en el menú izquierdo de la página de Colab.

Enseguida, buscar el siguiente modelo:
https://huggingface.co/dslim/bert-base-NER

In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [2]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"
example = "Google simplifies Generative AI Prohibited Use Policy"
example = "US Space Force launches expedited GPS mission using Falcon 9 rocket"

ner_results = nlp(example)

for r in ner_results:
  print(r)

{'entity': 'B-ORG', 'score': 0.99860626, 'index': 1, 'word': 'US', 'start': 0, 'end': 2}
{'entity': 'I-ORG', 'score': 0.99788636, 'index': 2, 'word': 'Space', 'start': 3, 'end': 8}
{'entity': 'I-ORG', 'score': 0.9989504, 'index': 3, 'word': 'Force', 'start': 9, 'end': 14}
{'entity': 'B-MISC', 'score': 0.98977906, 'index': 8, 'word': 'GPS', 'start': 34, 'end': 37}
{'entity': 'B-MISC', 'score': 0.9947201, 'index': 11, 'word': 'Falcon', 'start': 52, 'end': 58}
{'entity': 'I-MISC', 'score': 0.99607205, 'index': 12, 'word': '9', 'start': 59, 'end': 60}



# Sentiment Analysis

https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest

In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax


In [5]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
#model.save_pretrained(MODEL)
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [7]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")



1) negative 0.7236
2) neutral 0.2287
3) positive 0.0477


In [14]:
#Positive:
a ="Everything tasted delicious!"

# Negative:
b = "I'm fealling sick."

# Neutral:
c  = "The meeting lasted two hours and covered all the scheduled topics."

# Mixed:
d = "The movie had some great visuals, but the plot was confusing and poorly executed."

# Highly positive:
e = "This app has completely changed my life for the better—it's incredibly intuitive and reliable!"

In [15]:
text = [a,b,c,d,e]

def query(text, tokenizer, model):
  print("\ntext: ", text)
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)


  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [16]:
for t in text:
  query(t, tokenizer, model)


text:  Everything tasted delicious!
1) positive 0.9782
2) neutral 0.017
3) negative 0.0049

text:  I'm fealling sick.
1) negative 0.8545
2) neutral 0.1222
3) positive 0.0234

text:  The meeting lasted two hours and covered all the scheduled topics.
1) neutral 0.8093
2) positive 0.1858
3) negative 0.0049

text:  The movie had some great visuals, but the plot was confusing and poorly executed.
1) negative 0.8518
2) neutral 0.1285
3) positive 0.0197

text:  This app has completely changed my life for the better—it's incredibly intuitive and reliable!
1) positive 0.988
2) neutral 0.0074
3) negative 0.0045
